In [0]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
import random

In [0]:
colors = [2]

class Vehicle:
    
    def __init__(self, speed, road, position, nextRoad = None, speed_limit = 5, max_speed = 5, color = 0):
        self.speed = speed
        self.road = road
        self.nextRoad = nextRoad
        self.position = position
        self.length = 1
        self.speed_limit = speed_limit
        self.max_speed = max_speed
        self.color = random.choice(colors)
        
    def __repr__(self):
        return "Car's speed is {}, length {} and position {}.".format(self.speed, self.length, self.position)
    
    def adjustYourBehavior(self, numberOfFreeCells, red = False):
        self.numerOfFreeCells = numberOfFreeCells
        self.change_speed(numberOfFreeCells)
    
    def accelerate(self):
        if self.speed < self.max_speed:
          self.speed += 1
    
    def deccelerate(self):
        self.speed -= 1
    
    def urgentSlowDown(self):
        self.speed = 0

    def change_speed(self, numberOfFreeCells):
        if numberOfFreeCells <= self.max_speed:
          self.speed = numberOfFreeCells
        else:
          self.speed = self.max_speed 

    def randomisation(self):
        if np.random.rand() < 0.8:
            self.deccelerate()

    def move(self):
        self.position += self.speed
        
    def crossIntersectionRequest(self, entrance):
        if self.nextRoad.crossingAllowed(self.road):
            self.nextRoad.carsOnRoad.insert(entrance, self)
        pass



In [0]:
class Lights:
    def __init__(self, coordinate, red_time, green_time, phase = 'red', iterat = 0):
      self.coordinate = coordinate
      self.red_time = red_time
      self.green_time = green_time
      self.phase = phase
      self.iterat = iterat

    def increment_iter(self):
      self.iterat += 1

    def passing(self):
      counter = self.iterat
      if self.phase == 'red':
        while True:
          counter -= self.red_time
          if counter < 0:
            return False
          counter -= self.green_time
          if counter < 0:
            return True
      elif self.phase == 'green':
        while True:
          counter -= self.red_time
          if counter < 0:
            return True
          counter -= self.green_time
          if counter < 0:
            return False

In [0]:
class Road:
    def __init__(self, left, up, length, direction, maxSpeed, generator_probability = 0.5, lights = Lights(0, 0, 1), roadExit=0):
        #Coordinates
        self.left = left
        self.up = up
        self.length = length
        #east-west or north-south direction
        self.direction = direction
        self.maxSpeed = maxSpeed
        self.entrance = 0 # car generator or other road
        self.exit = roadExit # other road or 
        #Internal array for managing traffic
        self.road = []
        #Collection of cars currently on this road
        self.cars_on_road = []
        self.generator_probability = generator_probability
        #Lights location
        self.lights = lights
        

    def update_map(self, city):
        if self.direction == 'EW':
            self.road = np.flip(self.road)      
        if self.direction == 'SN':
            self.road = np.flip(self.road)      
        x = self.left
        y = self.up
        dist = 0
        while dist != self.length:
            if self.direction == 'SN' or self.direction == 'NS':
                city[x,y] = self.road[dist]
                x += 1
            elif self.direction == 'WE' or self.direction == 'EW':
                city[x,y] = self.road[dist]
                y += 1
            dist += 1
        return city

    def initialize_road_on_map(self, city):
        '''Inserts zeros into city map-array where the road is supposed to be'''
        self.road = np.zeros((self.length,), dtype=int)
        self.update_map(city)
        return city
    
    def read_map(self, city):
        x = self.left
        y = self.up
        self.road = []
        dist = 0
        while dist != self.length:
            if self.direction == 'SN' or self.direction == 'NS':
                self.road.append(city[x,y])
                x += 1
            elif self.direction == 'WE' or self.direction == 'EW':
                self.road.append(city[x,y])
                y += 1 
            dist += 1  
        if self.direction == 'EW':
            self.road = np.flip(self.road)      
        if self.direction == 'SN':
            self.road = np.flip(self.road)      

        return self.road     


    def update_road(self):
        self.road = np.zeros((self.length,), dtype=int)
        for car in self.cars_on_road:
            self.road[car.position] = 1
        if self.lights.passing():
            self.road[self.lights.coordinate] = 0
        else:
            self.road[self.lights.coordinate] = 1
    
    def rand_initialize_cars(self, probability):
        for i in range(1, 5):
            new_car_generated = np.random.random_sample()
            if new_car_generated > probability:
                position = i*2
                speed = 1
                v = Vehicle(speed, self, position)
                self.cars_on_road.append(v)
        self.update_road()


    def car_generator(self):
        test1 = np.random.rand()
        test2 = np.random.rand() + 1
        if test1 < self.generator_probability:
            self.cars_on_road.append(Vehicle(int(np.floor(test2)), self, 0))

    def sendout_car(self, car): 
        self.cars_on_road.remove(car)
                  

    def iteration(self):
        self.read_map(city)
        if self.road[0] == 0:
            self.car_generator()
        for car in self.cars_on_road:
            pred_position = car.position + car.speed
            if pred_position < len(self.road) - 1:  
                number_free_cells = 0
                for cell in range(car.position+1, pred_position + 2):
                    if self.road[cell] == 0:
                        number_free_cells += 1
                    else:
                        break
                car.adjustYourBehavior(number_free_cells, self.lights.passing())
            elif pred_position >= len(self.road) - 1:
                self.sendout_car(car)
            else:
                car.urgentSlowDown()
            car.move()
        
        self.update_road()
        self.update_map(city)
        self.lights.increment_iter()
    
    def __str__(self):
        return "Road's length is {}, max speed is {}, direction {} and cars probability {}. Crossings: {}".format(self.length, self.maxSpeed, self.direction, self.carsProbability, self.crossingRules)

In [0]:
def plot_city(city, roads):
    #IMPORT: from matplotlib import pyplot as plt
    clear_output(wait=True)
    fig = plt.figure()
    fig.set_size_inches(30, 10.5)
    for r in roads:
      if r.direction == 'WE':
        for c in r.cars_on_road:
          x = r.up+c.position
          y = r.left
          city[y,x] = c.color
      if r.direction == 'EW':
        for c in r.cars_on_road:
          x = r.up+r.length-1-c.position
          y = r.left
          city[y,x] = c.color
      if r.direction == 'NS':
        for c in r.cars_on_road:
          x = r.up
          y = r.left+c.position
          city[y,x] = c.color
      if r.direction == 'SN':
        for c in r.cars_on_road:
          x = r.up
          y = r.left+r.length-1-c.position
          city[y,x] = c.color
    plt.imshow(city, cmap='hot', interpolation='nearest')
    for r in roads:
      if r.direction == 'EW':
        for c in r.cars_on_road:
          plt.text(r.up+r.length-1-c.position, r.left, c.speed)
      elif r.direction == 'WE':
        for c in r.cars_on_road:
          plt.text(r.up+c.position, r.left, c.speed)
      elif r.direction == 'NS':
        for c in r.cars_on_road:
          plt.text(r.up, r.left+c.position, c.speed)
      elif r.direction == 'SN':
        for c in r.cars_on_road:
          plt.text(r.up, r.left+r.length-1-c.position, c.speed)
    plt.show()
    plt.pause(0.5)

In [0]:
time_steps = 100

#initialize an array representing city
vertical = 30
horizontal = 50 
city = np.array([[-1 for x in range(horizontal)] for y in range(vertical)])

#traffic lights
lights1 = Lights(24, 4, 7, phase = 'red')
lights2 = Lights(14, 8, 3, phase = 'green')

#create roads to be analyzed
roads = []
roads.append(Road(14,0,horizontal,'WE',5, 0.9))
roads.append(Road(16,0,horizontal,'EW',5, 0.9))
roads.append(Road(0,24,vertical,'NS',5, 0.3))
roads.append(Road(0,26,vertical,'SN',5, 0.3))

#initialization loop
for road in roads:
  road.initialize_road_on_map(city)
  road.read_map(city)
  road.update_map(city)

#simulation loop
for i in range(time_steps):
    for road in roads:
      road.iteration()

    print('Iteration: ' + str(i))
    plot_city(city, roads)